Example Kaggle notebook: https://colab.research.google.com/github/karthikraja95/fsdl_deforestation_detection/blob/master/fsdl_deforestation_detection/experimental/FSDL_Final_Model.ipynb#scrollTo=Fq9IBjOtUupg

In [3]:
! pip install torch torchvision torchaudio
! pip install -Uq kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchsummary import summary

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import pandas as pd
from pathlib import Path

## Download Dataset from Kaggle

1.  First set up the Kaggle API and download kaggle.json using these instructions: https://github.com/Kaggle/kaggle-api
2.   Run the following cell and upload kaggle.json


In [ ]:
from google.colab import files 
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download nikitarom/planets-dataset

In [ ]:
!unzip planets-dataset.zip

### Load the Dataset

In [ ]:
path = Path('./planet/planet')
train_df = pd.read_csv(path/'train_classes.csv')
train_df

## Prepare data for modeling

We know there are 17 total labels. Select the labels we think are most responsible for deforestation - agriculture, slash-burn, habitation, selective-logging, artisinal_mine, conventional_mine, cultivation

In [ ]:
deforestation_labels = ['agriculture', 'slash-burn', 'habitation', 'selective-logging',  'artisinal_mine', 'conventional_mine', 'cultivation']

Since we're focusing on deforestation/green, select conserved labels as well. 

In [ ]:
conserved_labels = ['primary']

### Combine these labels to create the binary class 'deforestation' - 0 for conserved, 1 if conservation is detected.

#### If any of the deforestation labels are found in an image's tags, deforestation = 1. Otherwise, deforestation = 0.

In [ ]:
train_df['deforestation'] = 0

tag = train_df['tags']
print(type(tag[0]))

deforested = tag.str.contains('|'.join(deforestation_labels))
train_df['deforestation'] = deforested.astype(int)
train_df


# for label in deforestation_labels:
#   for i in range(len(train_df)):
#     if train_df['deforestation'].iloc[i] == 1:
#       continue
#     if label in train_df['tags'].iloc[i].split():
#       train_df['deforestation'].iloc[i] = 1
     
display(train_df.head())

#if tags contains any label from deforestation_labels, it has a postive value of deforestation (1)

## Store images in a PyTorch Dataset


In [ ]:
! mkdir /content/planet/planet/data
! mv /content/planet/planet/train-jpg /content/planet/planet/data/train-jpg
! mkdir /content/planet/planet/validation
! mv /content/planet/planet/test-jpg /content/planet/planet/validation/test-jpg

In [ ]:
#We are using a binary classification model, so our image classes are 1 - significant deforestation; 0 - no significant deforestation
#Setting up transforms
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406], [0.225, 0.225, 0.225])])

#Set up dataset for training set and validation set 
train_data = datasets.ImageFolder('/content/planet/planet/data/', transform = data_transform)
val_data = datasets.ImageFolder('/content/planet/planet/validation', transform=data_transform)

#check to make sure number of images and df rows is the same
print('Training data images: ', len(train_data))
print('Training data csv: ', len(train_df))
print('Validation data images: ', len(val_data))

print(train_data)

# Set random seeds for reproducibility
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)

## Setting up dataloaders

In [ ]:
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

images, labels = iter(train_loader).next()
print(images.shape)
images = images.numpy()


### Plot the images in the batch

In [ ]:
fig = plt.figure(figsize=(15, 5))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size//2, idx+1, xticks=[], yticks=[])
    image = images[idx]
    image = image.transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.225, 0.225, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    ax.set_title("{}".format(train_df['deforestation'].iloc[idx]))

## Neural Network

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    #Input shape: (batch_size,3,256,256)
        
    # Convolutional 1 layer: 3x3 kernel, stride=1, padding=0, 2 output channels / feature maps
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=0)
    # Conv1 layer output size = (W-F+2P)/S+1 = ((256-3)/1)+1 = 254
    # Conv1 layer output shape for one image: [2,254,254]
    
    # Maxpool layer: kernel_size=2, stride=2
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    # Pool output shape for one image: ((254-2)/2)+1 = 127 [2,127,127]
    
    # Convolutional 2 layer: 3x3 kernel, stride=1, padding=0, 20 output channels / feature maps
    self.conv2 = nn.Conv2d(in_channels=2,out_channels=4,kernel_size=3, stride=1, padding=0)
    # Conv2 layer output size = (W-F+2P)/S+1 = ((127-3)/1)+1 = 125
    # Conv2 layer output shape for one image: [4,125,125]
    
    # Maxpool layer: kernel_size=2, stride=2
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    # Pool output shape for one image: ((125-2)/2)+1 = 62 (rounded down)  [4,62,62]
    
    # Input size: 4 * 62 * 62 = 15376  from pool2 pooling layer
    # 2 output channels (for the 2 classes)
    self.fc1 = nn.Linear(4*62*62, 2)
  
  def forward(self, x):
    # Two convolutional layers followed by relu and then pooling
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)

    # Flatten into a vector to feed into linear layer
    x = x.view(x.size(0), -1)
    
    # Linear layer
    x = self.fc1(x)
    return x


In [ ]:
net = ConvNet()

#display a summary of the layers of the model and output shape after each layer
summary(net, (images.shape[1:]), batch_size=batch_size, device='cpu')

### Define a cost/loss function and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

### Train the model

In [ ]:
def train_model(model, criterion, optimizer, train_loader, n_epochs, device):
  loss_over_time = [] # to track the loss as the network trains 
  model = model.to(device) # Send model to GPU if available
  model.train() # Set the model to training mode

  for epoch in range(n_epochs):  # loop over the dataset multiple times
      
      running_loss = 0.0
      
      for i, data in enumerate(train_loader):
          
          # Get the input images and labels, and send to GPU if available
          inputs, labels = data[0].to(device), data[1].to(device)

          # Zero the weight gradients
          optimizer.zero_grad()

          # Forward pass to get outputs
          outputs = model(inputs)

          # Calculate the loss
          loss = criterion(outputs, labels)

          # Backpropagation to get the gradients with respect to each weight
          loss.backward()

          # Update the weights
          optimizer.step()

          # Convert loss into a scalar and add it to running_loss
          running_loss += loss.item()
          
          if i % 1000 == 999:    # print every 1000 batches
              avg_loss = running_loss/1000
              # record and print the avg loss over the 1000 batches
              loss_over_time.append(avg_loss)
              print('Epoch: {}, Batch: {}, Avg. Loss: {:.4f}'.format(epoch + 1, i+1, avg_loss))
              running_loss = 0.0

  return loss_over_time

In [ ]:
#Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_epochs = 5
cost_path = train_model(net, criterion, optimizer, train_loader, n_epochs, device)

# visualize the loss
plt.plot(cost_path)
plt.xlabel('Batch (1000s)')
plt.ylabel('loss')
plt.show()

In [ ]:
### Validate the model on the validation set

def test_model(model,test_loader,device):
    # Turn autograd off
    with torch.no_grad():

        # Set the model to evaluation mode
        model = model.to(device)
        model.eval()

        # Set up lists to store true and predicted values
        y_true = []
        test_preds = []
        test_probs = []

        # Calculate the predictions on the test set and add to list
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            # Feed inputs through model to get raw scores
            logits = model.forward(inputs)
            # Convert raw scores to probabilities 
            probs = F.softmax(logits,dim=1)
            # Get discrete predictions using argmax
            preds = np.argmax(probs.cpu().numpy(),axis=1)
            # Add predictions and actuals to lists
            test_preds.extend(preds)
            test_probs.extend(probs)
            y_true.extend(labels.cpu().numpy())

        # Calculate the accuracy
        test_preds = np.array(test_preds)
        test_probs = np.array(test_probs)
        y_true = np.array(y_true)
        test_acc = np.sum(test_preds == y_true)/y_true.shape[0]
        
        # Recall for each class
        recall_vals = []
        for i in range(10):
            class_idx = np.argwhere(y_true==i)
            total = len(class_idx)
            correct = np.sum(test_preds[class_idx]==i)
            recall = correct / total
            recall_vals.append(recall)
    
    return test_acc, recall_vals,test_preds,test_probs

In [ ]:
# Calculate the test set accuracy and recall for each class
acc,recall_vals, preds, probs = test_model(net,val_loader,"cpu")
print('Validation set accuracy is {:.3f}'.format(acc))

print(preds)
print(probs)

In [ ]:
# Display a batch of predictions

with torch.no_grad():
    net = net.to(device)
    net.eval()
    # Get a batch of test images
    dataiter = iter(val_loader)
    images, labels = dataiter.next()
    images, labels = images.to(device), labels.to(device)
    # get predictions
    preds = np.squeeze(net(images).max(1, keepdim=True)[1].cpu().numpy())
    images = images.cpu().numpy()

# Plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size//2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(preds[idx], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx] else "red"))